In [1]:

from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
from opensearchpy.exceptions import TransportError, ConnectionError as OSCxnError

from requests_aws4auth import AWS4Auth
from openai import OpenAI, OpenAIError
from config import *
import time
import logging

from src.services.querying import llm_client, os_client, reconstruct_thread, answer_query, knn_search, Memory


In [2]:
query = "How much do we ususally charge for traspots for 'compassionate funerals'?"

hits, ids, q_vec = knn_search(query)

for i , hit in enumerate(hits, start=1):
    print(f"----Email thread {i}----")
    print()
    print(f"ID: {hit["_id"]}")
    print(f"Score: {hit["_score"]}")
    print(f"Subject: {hit["_source"]["subject"]}")
    print(f"Summary: {hit["_source"]["summary_text"]}")
    print()

----Email thread 1----

ID: t_autogen-java-a72b199e-9a80-4a1f-8135-c694957d734d@google.com
Score: 0.7728374
Subject: Redcoat Express Quotation
Summary: The email thread involves a communication from Redcoat Express Ltd to Compassionate Funerals, providing a quotation for air freight services costing £1,000 based on a chargeable weight of 100 kg. The quote details the service, charges, and terms, with an indication that it is based on the information provided, and includes a request for acceptance and signatures.

----Email thread 2----

ID: t_cajraaq8den7akjznr1eyqmvabovylsnbat+-eh7b+7d=vsn5jw@mail.gmail.com
Score: 0.7653014
Subject: Flight to Banju
Summary: Hasina requests quotes and scheduling details for next week, providing her contact information and background on Compassionate Funerals' services. She confirms a weight of 120kg for a shipment and later states the cost as GBP1600. The thread involves coordinating funeral-related arrangements, including timing, weight, and pricing.


In [3]:
retrieved_ids = []
memory = Memory(
    llm_client=llm_client,
    os_client=os_client,
    short_term_tokens=1000,
    mid_term_turns=3,
    memory_model=MEMORY_MODEL,
    embeddings_model=EMBEDDINGS_MODEL
)

In [21]:
bank=[
 "What how does Redcoat work with Lufthansa?",
 "Can you give specific examples?",
 "What are Lufthansa's rates to Africa?",
 "And what about other destinations, like US?",
 "Who is Simon and his role?",
 "How does he collaborate with Sarah Hawkins?",
 "If we were to compare US and African imports to the UK. How do they compare?"
 "Draft an SOP to import goods to the UK."
 ]

query = query = "How much do we ususally charge for traspots for 'compassionate funerals'?" #bank[2]
retrieved_ids = []  

prompt, response, mem_ctx, retrieved_ids, q_vec = answer_query(query, retrieved_ids=retrieved_ids, memory=memory)




----Rewritted query-----
 What are the typical charges for transporting cargo for "compassionate funerals"? 
----Rewritted query-----
 


In [22]:
import tiktoken

enc = tiktoken.encoding_for_model(QUERY_MODEL)

print(len(enc.encode(prompt)))

2785


In [23]:
print(response)

Based on the information provided in the email threads, Redcoat Express Ltd usually charges around £10.00 per kilogram for air freight services for "compassionate funerals." Here are some specific examples:

1. In an email dated November 2, 2023, Redcoat Express Ltd provided a quotation to Compassionate Funerals for air freight services with a chargeable weight of 100 kg, costing £1,000. This indicates a rate of £10.00 per kilogram (£1,000 total cost for 100 kg).

2. In another email dated April 8, 2022, Compassionate Funerals requested a quote for transporting 120 kg of cargo, to which Redcoat Express Ltd responded with a cost of GBP1600. This aligns with the approximate rate of £10.00 per kilogram (£1,600 total cost for 120 kg).

3. Additionally, in a separate email dated May 7, 2024, Redcoat Express Ltd provided a detailed air freight quote of £1,650 for transporting 120 kg of cargo for Compassionate Funerals. This further supports the typical rate of around £10.00 per kilogram (£1,

Conditional Retrieval

Cooldown TLL

Freshness Boosting

MRR (Penalizing documents which are too similar to what we already have)

Hierarchical Memory